In [28]:
%matplotlib inline
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm 
import sklearn.linear_model as lm 
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer

In [29]:
#classe qui a pour but d'afficher les objets de type dataframe
class display(object):
    """Affichage d'une représentation HTML des objets"""
    
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [30]:
df_test = pd.read_csv("application_test.csv")
df_train = pd.read_csv("application_train.csv")

In [31]:
df_train = df_train[['TARGET', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'DEF_30_CNT_SOCIAL_CIRCLE']]
test = df_test[['EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'DEF_30_CNT_SOCIAL_CIRCLE']]
df_train

,TARGET,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,DEF_30_CNT_SOCIAL_CIRCLE
0,1,0.139376,0.083037,0.262949,-637,-9461,2,2,-1134.0,-2120,2.0
1,0,NaN,0.311267,0.622246,-1188,-16765,1,1,-828.0,-291,0.0
2,0,0.729567,NaN,0.555912,-225,-19046,2,2,-815.0,-2531,0.0
3,0,NaN,NaN,0.650442,-3039,-19005,2,2,-617.0,-2437,0.0
4,0,NaN,NaN,0.322738,-3038,-19932,2,2,-1106.0,-3458,0.0
...,...,...,...,...,...,...,...,...,...,...,...
50455,0,0.529890,0.157266,0.232957,-2568,-11055,3,3,0.0,-1765,1.0
50456,0,0.593718,NaN,0.559591,365243,-20331,2,2,-353.0,-1754,0.0
50457,0,NaN,0.841934,0.713940,365243,-21646,2,2,-1144.0,-4448,0.0
50458,0,NaN,0.583988,0.590705,-1828,-15230,3,3,-1525.0,-3969,0.0


In [32]:
df_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_test.columns

Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=121)

In [34]:
df_train.isna().sum()

TARGET                             0
EXT_SOURCE_3                   10027
EXT_SOURCE_1                   28417
EXT_SOURCE_2                     128
DAYS_EMPLOYED                      0
DAYS_BIRTH                         0
REGION_RATING_CLIENT_W_CITY        0
REGION_RATING_CLIENT               0
DAYS_LAST_PHONE_CHANGE             2
DAYS_ID_PUBLISH                    0
DEF_30_CNT_SOCIAL_CIRCLE         173
dtype: int64

In [47]:
df_train = pd.get_dummies(df_train)
df_train= df_train.fillna(method = 'ffill')


In [48]:
X = df_train.loc[:, df_train.columns != 'TARGET']
y = df_train['TARGET']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [38]:
print(X_train, y_train)

       EXT_SOURCE_3  EXT_SOURCE_1  EXT_SOURCE_2  DAYS_EMPLOYED  DAYS_BIRTH  \
26654      0.445396           NaN      0.330060           -246      -11145   
37664      0.656158           NaN      0.719653          -1656      -13570   
40170      0.309275      0.699782      0.560668          -2973      -15179   
12249           NaN           NaN      0.367620          -2459      -12546   
5151            NaN           NaN      0.551091         365243      -19600   
...             ...           ...           ...            ...         ...   
22857      0.408359      0.348431      0.029127          -3610      -13894   
39965      0.576209      0.381194      0.697180          -3157      -14884   
20803      0.602386      0.465618      0.159779         365243      -17701   
46391      0.740799           NaN      0.339934          -4272      -14700   
13394      0.680139      0.501574      0.736800           -298      -14400   

       REGION_RATING_CLIENT_W_CITY  REGION_RATING_CLIENT  \
266

In [50]:
X_train.isna().sum().sum()


0

In [40]:
imputer = IterativeImputer()
X.isna().sum()

EXT_SOURCE_3                   10027
EXT_SOURCE_1                   28417
EXT_SOURCE_2                     128
DAYS_EMPLOYED                      0
DAYS_BIRTH                         0
REGION_RATING_CLIENT_W_CITY        0
REGION_RATING_CLIENT               0
DAYS_LAST_PHONE_CHANGE             2
DAYS_ID_PUBLISH                    0
DEF_30_CNT_SOCIAL_CIRCLE         173
dtype: int64

In [27]:
X = pd.DataFrame(imputer.fit_transform(X), columns = X.columns)
X

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50455,158420.0,1.0,76500.0,1185120.0,39298.5,900000.0,0.018029,-11055.0,-2568.0,-5677.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
50456,158421.0,0.0,63000.0,508495.5,21672.0,454500.0,0.025164,-20331.0,365243.0,-699.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
50457,158422.0,0.0,112500.0,1350000.0,37255.5,1350000.0,0.008474,-21646.0,365243.0,-5659.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
50458,158423.0,0.0,157500.0,238500.0,15655.5,238500.0,0.018209,-15230.0,-1828.0,-8529.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [51]:
model = RandomForestClassifier(max_depth=3,random_state=0,n_estimators=50)
model = model.fit(X_train,y_train)

y_pred_test = model.predict(X_test)

In [52]:
accuracy_score(y_test, y_pred_test)

0.9200096084554408